In [1]:
import tensorflow as tf
import numpy as np
import tempfile

In [2]:
batchSize = 5
maxNumSteps = 8
embeddingSize = 3
nNeurons = 4
vocabulary_size = 10
initEmbeddings = np.random.randn(vocabulary_size, embeddingSize)

In [34]:
tf.reset_default_graph()
inputTokens = tf.placeholder(tf.int32, [batchSize, maxNumSteps])
inputLens = tf.placeholder(tf.int32, [batchSize])
targets = tf.placeholder(tf.float64, [batchSize])
rnnCell = tf.nn.rnn_cell.BasicRNNCell(nNeurons)
initState = rnnCell.zero_state(batchSize, tf.float64)

embedding = tf.Variable(initEmbeddings, name='inputEmbeddings', trainable=False, dtype=tf.float64)
inputData = tf.nn.embedding_lookup(embedding, inputTokens)

raw_outputs, last_states = tf.nn.dynamic_rnn(
    cell=rnnCell,
    dtype=tf.float64,
    sequence_length=inputLens,
    inputs=inputData)
    
flattened_outputs = tf.reshape(raw_outputs, [-1, nNeurons])
index = tf.range(0, batchSize) * maxNumSteps + inputLens - 1
outputs = tf.gather(flattened_outputs, index)
outputW = tf.get_variable("outputW", [nNeurons, 1], dtype=tf.float64)
outputB = tf.get_variable("outputB", [1], dtype=tf.float64)
prediction = tf.add(tf.matmul(outputs, outputW), outputB)
loss = tf.reduce_sum(tf.pow(prediction-targets, 2)/batchSize)

lr = tf.Variable(0.1, trainable=False)
tvars = tf.trainable_variables()
tvars = np.take(tvars, [0,2,3]).tolist() # exclude RNN bias from training
optimizer = tf.train.GradientDescentOptimizer(lr)
learningStep = optimizer.minimize(loss, var_list=tvars)
initAll = tf.initialize_all_variables()

In [40]:
# data = np.random.randn(batchSize, maxNumSteps, embeddingSize)
data = np.random.randint(0, high=vocabulary_size-1, size=(batchSize, maxNumSteps))
lens = [maxNumSteps for i in xrange(batchSize)]
t = np.random.randn(batchSize)
with tf.Session() as sess:
    sess.run(initAll)
#     e = sess.run(embedding)
#     print e
#     print data
#     s = sess.run(inputData, feed_dict={inputTokens:data, inputLens:lens, targets:t})
#     print s
#     l = sess.run(loss, feed_dict={inputTokens:data, inputLens:lens, targets:t})
#     print l
#     for v in tf.trainable_variables():
#         print v.name
#         print sess.run(v)
#     w = sess.run(outputW)
#     b = sess.run(outputB)
#     print w
#     print b
#     r = sess.run(raw_outputs, feed_dict={inputTokens:data, inputLens:lens, targets:t})
#     print len(r), len(r[0])
#     print r[:,-1]
#     i = sess.run(index, feed_dict={inputTokens:data, inputLens:lens, targets:t})
#     print i
#     o = sess.run(outputs, feed_dict={inputTokens:data, inputLens:lens, targets:t})
#     print o
#     m = sess.run(tf.matmul(outputs, outputW), feed_dict={inputTokens:data, inputLens:lens, targets:t})
#     print m
#     p = sess.run(prediction, feed_dict={inputTokens:data, inputLens:lens, targets:t})
#     print p
    sess.run(learningStep, feed_dict={inputTokens:data, inputLens:lens, targets:t})
    l = sess.run(loss, feed_dict={inputTokens:data, inputLens:lens, targets:t})
    print l
    for v in tf.trainable_variables():
        print v.name
        print sess.run(v)

[[-0.80577191  0.52229679  0.93380249]
 [-0.84560716 -0.43266551  1.09092011]
 [-0.2683122  -0.23485634  0.75907219]
 [-0.24423481  0.58242155  0.9500426 ]
 [ 0.18173485 -0.04901824 -0.57453806]
 [-0.30975785  0.99402065 -1.20278396]
 [ 0.23538442  0.08246597  1.19941157]
 [ 0.65812436 -1.68136343  0.99296283]
 [-0.28339739 -3.33845653 -0.54183886]
 [-1.19780826 -0.36035379  0.67650091]]
[[5 6 0 7 7 1 6 8]
 [0 1 8 0 4 1 8 2]
 [4 3 5 0 1 7 3 6]
 [6 1 8 0 7 8 4 7]
 [5 6 6 2 4 3 8 8]]
[[[-0.30975785  0.99402065 -1.20278396]
  [ 0.23538442  0.08246597  1.19941157]
  [-0.80577191  0.52229679  0.93380249]
  [ 0.65812436 -1.68136343  0.99296283]
  [ 0.65812436 -1.68136343  0.99296283]
  [-0.84560716 -0.43266551  1.09092011]
  [ 0.23538442  0.08246597  1.19941157]
  [-0.28339739 -3.33845653 -0.54183886]]

 [[-0.80577191  0.52229679  0.93380249]
  [-0.84560716 -0.43266551  1.09092011]
  [-0.28339739 -3.33845653 -0.54183886]
  [-0.80577191  0.52229679  0.93380249]
  [ 0.18173485 -0.04901824 -0.5

In [33]:
np.dot(r[:,-1], w)

array([[ 0.66335526],
       [ 0.70219608],
       [-0.58211488],
       [ 0.07739599],
       [ 0.10125138]])

TypeError: list indices must be integers, not list